<p style="font-weight:bold;"> <span style="font-size: 36px"> Interact With Import Scope Calculation</span> </p>

<br><br>
After successfully loading the parsed data in the dataSource, the import scopes for a specific partition can be re-evaluated in this notebook. 
The import scopes can then be displayed, examined and used for further calculations.

# Basic Initialization

In [0]:
#!import "../../Constants/CalculationEngine"

In [0]:
#!eval-notebook "../../Initialization/InitSystemorphBaseToMemory"
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.InitializeFrom(DataSource);

# Import of parsed data

In [0]:
Debug.Enable = true;

In [0]:
ProjectionCount = 5;

In [0]:
await Import.FromFile("../../Files/TransactionalData/NominalCashflows_CH_2020_12_AyLRC.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()

# Args

## Primary Args

In [0]:
var reportingNode = "CH";
var year = 2020;
var month = 12;
var importFormat = ImportFormats.Cashflow;
var scenario = (string)null;

ImportArgs args = new ImportArgs(reportingNode, year, month, default(Periodicity), scenario, importFormat);
args

## Secondary Args

In [0]:
var allArgs = await GetAllArgsAsync(args, DataSource, ImportFormats.Cashflow);
allArgs

In [0]:
// var workspaceToCompute = Workspace.CreateNew();
// workspaceToCompute.InitializeFrom(DataSource);

In [0]:
// await ComputeAsync(args, Workspace, workspaceToCompute, true)

# Construct ImportStorage

In [0]:
var storage = new ImportStorage(args, DataSource, Workspace);
await storage.InitializeAsync();

# Create universe and Identities

In [0]:
var universe = Scopes.ForStorage(storage).ToScope<IModel>();
var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);

In [0]:
identities.First()

In [0]:
var idByDn = identities.ToDictionaryGrouped(x => x.DataNode, x => x.ToArray());

In [0]:
var ivs = Scopes.ForIdentities(identities, storage).ToScopes<ComputeAllScopes>().SelectMany(x => x.CalculatedIfrsVariables).ToArray();

In [0]:
int projection = 0;

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<PvToIfrsVariable>().SelectMany(x => x.PvLocked.Concat(x.PvCurrent)).ToArray();

In [0]:
Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = 0}), storage).ToScopes<PvToIfrsVariable>().SelectMany(x => x.PvLocked.Concat(x.PvCurrent)).ToArray()
.Where(x => x.EconomicBasis=="L" && x.Values.Any(y => Math.Abs(y)> Precision) && x.DataNode == "DT1.1" && new[]{"PR","NIC"}.Contains(x.AmountType))

.Where(x=> x.AmountType=="NIC")//.Aggregate()

In [0]:
Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = 1}), storage).ToScopes<PvToIfrsVariable>().SelectMany(x => x.PvLocked.Concat(x.PvCurrent)).ToArray()
.Where(x => x.EconomicBasis=="L" && x.Values != null && x.Values.Any(y => Math.Abs(y)> Precision) && x.DataNode == "DT1.1" && new[]{"PR","NIC"}.Contains(x.AmountType))

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<NominalToIfrsVariable>().SelectMany(x => x.CumulatedNominal).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<RaToIfrsVariable>().SelectMany(x => x.RaCurrent.Concat(x.RaLocked)).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<ActualToIfrsVariable>().SelectMany(x => x.Actual.Concat(x.AdvanceActual).Concat(x.OverdueActual)).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<DeferrableToIfrsVariable>().SelectMany(x => x.Deferrable).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<EaForPremiumToIfrsVariable>().SelectMany(x => x.BeEAForPremium.Concat(x.ActEAForPremium)).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<TmToIfrsVariable>().SelectMany(x => x.AmortizationFactor.Concat(x.Csms).Concat(x.Loss)).ToArray();